In [ ]:
from espuma import Case_Directory, Boundary_Probe
from espuma_utils import get_profile_over_depth, get_stacked_biomass
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator, MaxNLocator
import pyvista as pv

In [ ]:
try:
    NOTEBOOK_NAME = __file__
except NameError:
    NOTEBOOK_NAME = __vsc_ipynb_file__ #noqa: F821 #type: ignore

colors = {
    'active': "#ff3146", 
    'inactive':"#4f4b7e",
    'labile': "#5470b8",
    'recalcitrant': "#4f4b7e"}

In [ ]:
pv.start_xvfb()
pv.global_theme.trame.server_proxy_enabled = True
pv.set_jupyter_backend("static")

plt.style.use('./matplotlib_style/edwin.mplstyle')

In [ ]:
## Bring the template to clone
of_template = Case_Directory("template")

## Check the value of $\kappa$ and $\rho_X$from the template
print(f"{of_template.constant.transportProperties['diffusiveGrowth'] = }")
print(f"{of_template.constant.transportProperties['rho_X'] = }")

## Make a folder to store the run cases
(cases_folder := Path("CASES_fit_experimental")).mkdir(exist_ok=True)

In [ ]:
def set_up_and_run(kappa:float, rho_x: float):
    
    case_name = f"rhox_{rho_x:04}__kappa_{kappa:5.2e}"
    max_time = 23 * 86_400

    try:
        of_case = Case_Directory.clone_from_template(of_template, cases_folder / case_name)
    
    except OSError:
        of_case = Case_Directory(cases_folder / case_name)

    ## If the case was already ran, just return it
    if of_case.list_times[-1] >= max_time:
        return of_case
    
    of_case.system.controlDict["endTime"] = max_time
    of_case.constant.transportProperties["rho_X"] = f"rho_X [ 1 -3 0 0 0 0 0 ] {rho_x}"
    of_case.constant.transportProperties["kappa"] = f"rho_X [ 1 -3 0 0 0 0 0 ] {kappa}"

    of_case._blockMesh()
    of_case._runCase()

    return of_case

In [ ]:
of_case = set_up_and_run(kappa=(kappa:=1e-11), rho_x=(rho_x:=10))

In [ ]:
profile = get_profile_over_depth(of_case, t_target_days=23)
profile

In [ ]:
from pandas import read_excel
from pathlib import Path

In [ ]:
# 🏁 End of experiment results
EXPERIMENT_PATH = Path(
    "../experiments/Rosenzweig_2011/experimentalData/.hiddendata"
)

cfu = read_excel(EXPERIMENT_PATH / "Live-counts.xlsx", sheet_name="Inoculated")
protein = read_excel(
    EXPERIMENT_PATH / "Protein-bradfordMethod.xlsx", sheet_name="Inoculated"
)
theta = read_excel(EXPERIMENT_PATH / "theta-endexperiment.xlsx", sheet_name='Inoculated')

In [ ]:
cfu

In [ ]:
protein

In [ ]:
theta

In [ ]:
COLUMN_LENGHT = 0.60

fig_profiles, axs = plt.subplots(
    1, 2, figsize=[6, 4], sharey=True, gridspec_kw=dict(wspace=0.1)
)

## Final active biomass over depth
ax = axs[0]
(l,) = ax.plot(
    profile["XAR"],
    profile["Distance"] - COLUMN_LENGHT,
    label="Simulation",
    c=colors['active'],
    alpha=0.9,
    lw=3,
    ls=(0, (4,0.7)),
)
ax.set_xscale("log")
ax.set_xlim(2e-6, 2e-0)
ax.set_ylim(bottom= -COLUMN_LENGHT)
ax.set_ylabel("Depth   $z$ [m]")
ax.set_xlabel(R"Active biomass [g/L]")
ax.xaxis.set_major_locator(LogLocator())
ax.xaxis.set_minor_locator(LogLocator(subs="auto"))
ax.set_title("  a", loc='left', y=0.9, weight=700)

ax2 = ax.twiny()
ax2.errorbar(
    cfu["CFU/mL"], 
    cfu["z (m)"] - COLUMN_LENGHT, 
    xerr=cfu["stdev"], fmt="none", ecolor="gray", capsize=2
)
(l2,) = ax2.plot(
    cfu["CFU/mL"],
    cfu["z (m)"] - COLUMN_LENGHT,
    label=R"Experimental",
    marker="o",
    markersize=5,
    mfc=colors['active'],
    c="gray",
    lw=1,
    ls="dashed",
)
ax2.set_xscale("log")
ax2.set_ylim(ax.get_ylim())
ax2.spines.top.set_visible(True)
ax2.set_xlim(5e6, 5e10)
ax2.set_xlabel("Microbial counts\n[CFU/mL]")
ax2.legend(
    [l, l2],
    [l.get_label(), l2.get_label()],
    title=R"$X_{\mathsf{aerobic\;resp.}}$",
    title_fontproperties=dict(size=10, weight=100),
    loc="lower right",
)
ax2.spines.top.set_position(("axes", 1.05))

## Final inactive biomass over depth
ax = axs[1]
(l,) = ax.plot(
    profile["EPS"] + profile["XI"],
    profile["Distance"] - COLUMN_LENGHT,
    label="Simulation",
    c=colors['inactive'],
    alpha=0.9,
    lw=3,
    ls="dashed",
)

DENSITY_SAND = 1_530  # g/L_REV
MG_TO_G = 1000
PROTEIN_PERCENT_IN_EPS = 0.678  # See https://doi.org/10.1016/j.biortech.2009.01.053

(l2,) = ax.plot(
    protein["mg protein/g dry sand"]
    * DENSITY_SAND
    / MG_TO_G
    / PROTEIN_PERCENT_IN_EPS,  ##<- convert to g/L_REV
    protein["z (m)"] - COLUMN_LENGHT,
    label=R"Experimental",
    marker="o",
    markersize=5,
    c="gray",
    lw=1,
    ls="dashed",
    mfc=colors['inactive'],
)

ax.legend(
    [l, l2],
    [l.get_label(), l2.get_label()],
    title=R"$X_{\mathsf{labile}} + X_{\mathsf{recalcitrant}}$",
    title_fontproperties=dict(size=10, weight=100),
    loc="lower right",
)

ax.set_xscale("log")
ax.set_xlim(2e-3, 10e-0)
# ax.set_ylim(bottom=-COLUMN_LENGHT)
ax.set_xlabel(R"Inactive biomass  [g/L]")
ax.xaxis.set_major_locator(LogLocator())
ax.xaxis.set_minor_locator(LogLocator(subs="all"))
ax.set_title("  b", loc='left', y=0.9, weight=700)

plt.savefig(
    f"plots/{NOTEBOOK_NAME.rpartition('/')[-1].removesuffix('.ipynb')}(profile_experimental).pdf", 
    format="pdf", bbox_inches='tight', pad_inches=0.05
)

plt.show()

In [ ]:
h = profile["h"]
Sw = profile["Sw"]

fig_profiles, axs = plt.subplots(
    1, 2, figsize=[6, 4], sharey=True, gridspec_kw=dict(wspace=0.1)
)

## Final water content over depth
axs[0].plot(h, profile["Distance"] - COLUMN_LENGHT, label="h(z)", c='k')
axs[0].set_xlabel(R"Matric head   $h$ [m]")
axs[0].set_ylabel("Depth   $z$ [m]")

axs[1].plot(Sw, profile["Distance"] - COLUMN_LENGHT, label="Sw(h)", c='tab:blue')
axs[1].set_xlabel(R"Water saturation   $S_w$ [-]")

for ax in axs:
    ax.legend()
    ax.set_ylim(bottom= -COLUMN_LENGHT)

plt.show()

In [ ]:
import numpy as np

α = 2.79
θs = 0.385
θr = 0.012
n = 7.26
m = 1 - 1 / n

def vanGenuchten(h: np.array) -> np.array:
    global n, m , α
    θe = np.where(h >= 0, 1.0, np.power(1 + np.power(α * h * np.sign(h), n), -m))
    return θe


def waterSaturation(h: np.array) -> np.array:
    global θs, θr
    θe = vanGenuchten(h)
    θ = θe * (θs - θr) + θr
    return θ

In [ ]:
fig, ax = plt.subplots(
    1, 1, figsize=[6, 4], sharey=True, gridspec_kw=dict(wspace=0.1)
)

ax.plot(waterSaturation(h), profile["Distance"] - COLUMN_LENGHT, label="Unmodified", lw=3, c='purple')

for i in np.arange(7.5, 1.5, -1):
    n = i
    m = 1 - 1 / n
    theta_new_method = waterSaturation(h)
    ax.plot(theta_new_method, profile["Distance"] - COLUMN_LENGHT, label=f"{n = :.2f}", lw=1)

ax.legend()
ax.set_ylim(bottom= -COLUMN_LENGHT)
ax.set_ylabel("Depth   $z$ [m]")
ax.set_xlabel(R"Water content   $\theta$ [-]")

plt.show()

In [ ]:
## Recalculated water content from EPS affected parameters
n = 3.5
m = 1 - 1 / n
theta_new_method = waterSaturation(h)

## Water content from biomass
rho_x = 10
fraction_water_biomass = 0.80
porosity_0 = 0.385
porosity = profile["porosity"]
sum_biomass = porosity_0 - porosity
n_biomass = fraction_water_biomass * sum_biomass/rho_x

fig_profiles, ax = plt.subplots(
    1, 1, figsize=[6, 4], sharey=True, gridspec_kw=dict(wspace=0.1)
)

ax.plot(theta_new_method + n_biomass, profile["Distance"] - COLUMN_LENGHT, label="My calculator", lw=2, ls="dashed", c='tab:red')
ax.scatter(theta['θ'], theta["z (m)"] - COLUMN_LENGHT)

ax.legend()
ax.set_ylim(bottom= -COLUMN_LENGHT)
ax.set_ylabel("Depth   $z$ [m]")
ax.set_xlabel(R"Water content   $\theta$ [-]")

plt.show()

In [ ]:
prb = Boundary_Probe(of_case, of_case.system.boundaryProbes)
xarr = prb.array_data
sb = get_stacked_biomass(of_case)
time = [t/86400 for t in of_case.list_times]

In [ ]:
cross_area_column = float(of_case.system.blockMeshDict["diameter"]) ** 2
print(f"{cross_area_column = :.3f} m²")

In [ ]:
fig, (qax,ax) = plt.subplots(2, 1, figsize=(5,4), sharex=True, height_ratios=[1,2], gridspec_kw=dict(hspace=0.3))

# Extract integrated variables
xar =  [v["XAR"][0]/cross_area_column for v in sb.values()]
xeps = [v["EPS"][0]/cross_area_column for v in sb.values()]
xin =  [v["XI"][0]/cross_area_column  for v in sb.values()]

ax.stackplot(
    time, 
    *(xar, xeps, xin),
    labels=(R"$X_{\mathsf{aerobic\;resp.}}$", R"$X_{\mathsf{labile}}$", R"$X_{\mathsf{recalcitrant}}$"),
    colors = (colors['active'], colors['labile'], colors['recalcitrant']),
    # alpha=0.8
)
    
ax.set_xlabel("Time $t$ [d]")
ax.set_xlim(left=0)
ax.legend(loc="upper left", handlelength=1, handletextpad=0.3, handleheight=1)
ax.ticklabel_format(axis='y', useMathText=True, scilimits=(0,0))
ax.yaxis.set_major_locator(MaxNLocator(3))

ax.set_ylim(bottom=0, top=8e-2)
ax.set_xlim(0, 23)
ax.set_ylabel("Total biomass\n" + R"$\int{X_j} \; dz$ [kg/m²]")
# ax.text(1.0, 0.85, 
#     "Always wet", 
#     ha='right', va='top', transform=ax.transAxes,
#     weight=200
# )
ax.set_title("b", loc='right', y=0.9, weight=700)

qax.plot(xarr.time/86400, xarr.isel(probes=0).Uz, lw=1)
qax.set_ylabel("Influx \n" R"$q_{\mathsf{top}}$ [m/s]")
qax.ticklabel_format(axis='y', useMathText=True)
qax.yaxis.set_major_locator(MaxNLocator(4))
qax.set_title("a", loc='right', weight=700)
plt.savefig(
    f"plots/{NOTEBOOK_NAME.rpartition('/')[-1].removesuffix('.ipynb')}(over_time).pdf", 
    format="pdf", bbox_inches='tight', pad_inches=0.05
)
plt.show()
    

In [ ]:
total_x_end = xar[-1] + xeps[-1] + xin[-1]
print(
    f"""
    Proportions at the end of the experiment were:
    XAR = {100*xar[-1]/total_x_end:.1f}
    EPS = {100*xeps[-1]/total_x_end:.1f}
    XIN = {100*xin[-1]/total_x_end:.1f}
    """
)

In [ ]:
print(
    f"{of_case.constant.biochemicalProperties['fd'] = }\n"
    f"{xeps[-1]/(xin[-1] + xeps[-1]) = :.2f}"
)

In [ ]:
from math import pi

## real cross section column = diameter**2 * pi / 4
real_cross_area_column = 0.08**2 * pi / 4  # m²

print(
    f"Real experiment cross sectional area = {real_cross_area_column:.6f} m²\n"
    f"In the model, cross sectional area   = {cross_area_column:.6f} m²"

)


In [ ]:
q_ini = float(xarr.isel(probes=0).Uz[0])  # m/s
q_ini *= real_cross_area_column               # m/s * m² -> m³/s
q_ini *= 60 * 1000 * 1000                     # m³/s -> mL/min

q_end = float(xarr.isel(probes=0).Uz[-1]) # m/s
q_end *= real_cross_area_column               # m/s * m² -> m³/s
q_end *= 60 * 1000 * 1000                     # m³/s -> mL/min

print(
    f"""
    Initial flowrate: {q_ini:.3f} mL/min
    End flowrate: {q_end:.3f} mL/min 
    -------------------------------------
    Ratio: {q_end/q_ini = :.4f}
    """
)
